In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen

from bs4 import BeautifulSoup
import requests

In [3]:
url = "https://www.w3schools.com/asp/default.asp"

# Lets make a request to "https://www.w3schools.com/html/default.asp"
# Will store the result in 'res' variable
page = requests.get(url)

txt = page.text
status = page.status_code

In [4]:
print(status)

200


In [5]:
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
heading_tags = ["h1","h2","h3"]
for tags in soup.find_all(heading_tags):
    print(tags.name + ' -> ' + tags.text.strip())

h2 -> Tutorials
h3 -> HTML and CSS
h3 -> Data Analytics
h3 -> XML Tutorials
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> Web Building
h3 -> Data Analytics
h3 -> XML Tutorials
h2 -> References
h3 -> HTML
h3 -> CSS
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> XML
h3 -> Character Sets
h2 -> Exercises and Quizzes
h3 -> Exercises
h3 -> Quizzes
h3 -> Courses
h3 -> Certificates
h2 -> ASP Tutorials
h2 -> WP Tutorial
h2 -> ASP.NET Razor
h2 -> ASP Classic
h2 -> ASP Reference
h2 -> ADO Tutorial
h2 -> ADO Objects
h1 -> ASP and ASP.NET Tutorials
h2 -> The ASP Technology
h2 -> Classic ASP - Active Server Pages
h2 -> ASP.NET
h2 -> ASP.NET Web Pages
h2 -> ASP.NET MVC
h2 -> ASP.NET Web API
h2 -> ASP.NET Web Forms
h2 -> ASP.NET Core
h2 -> Report Error
h2 -> Thank You For Helping Us!


In [7]:
heading1 = soup.find_all('h2', class_ = 'left')
heading1

[<h2 class="left">ASP Tutorials</h2>,
 <h2 class="left">WP Tutorial</h2>,
 <h2 class="left">ASP.NET Razor</h2>,
 <h2 class="left">ASP Classic</h2>,
 <h2 class="left">ASP Reference</h2>,
 <h2 class="left">ADO Tutorial</h2>,
 <h2 class="left">ADO Objects</h2>]

In [8]:
sub_headings = []
for i in heading1:
    print(i.text)
    sub_headings.append(i.text)

ASP Tutorials
WP Tutorial
ASP.NET Razor
ASP Classic
ASP Reference
ADO Tutorial
ADO Objects


In [9]:
heading2 = soup.find_all('a', target='_top')
heading2

[<a class="w3-bar-item w3-button" href="/html/html_quiz.asp" target="_top">HTML Quiz</a>,
 <a class="w3-bar-item w3-button" href="/css/css_quiz.asp" target="_top">CSS Quiz</a>,
 <a class="w3-bar-item w3-button" href="/js/js_quiz.asp" target="_top">JavaScript Quiz</a>,
 <a class="w3-bar-item w3-button" href="/sql/sql_quiz.asp" target="_top">SQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/mysql/mysql_quiz.asp" target="_top">MySQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/php/php_quiz.asp" target="_top">PHP Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/python_quiz.asp" target="_top">Python Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/numpy/numpy_quiz.asp" target="_top">NumPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/pandas/pandas_quiz.asp" target="_top">Pandas Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/scipy/scipy_quiz.php" target="_top">SciPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/jquery/jquery_quiz.asp" 

In [10]:
# getting the Headings, Sub-Headings, Links and Contents

# CREATING RELEVANT VARIABLES
tag = ''
page_table = []
course_heading = []
course_subheading = []
course_links = []
sub_page_content_list = []
unordered_lists = []
ordered_lists = []
list_headings = []
examples_per_page = []
excercises = []
base_url = "https://www.w3schools.com/asp/"
course_url = "https://www.w3schools.com/asp/"

# FINDING ALL THE LINKS PRESENT IN THE LEFT MENU AND ITERATING OVER THE LINKS TO SCRAPE THE DATA
test = soup.find('div', id='leftmenuinnerinner')

for i in test:
    if 'h2' in str(i):
        tag = i.text
    else:
        if (tag!='') and ('a href' in str(i)) and ('div class="tut_overview"' not in str(i)):
            course_heading.append(tag)
            course_subheading.append(i.text)
            extra = i.get('href')
           
            if extra is not None :
                print(extra)
                if extra[0] == '/':
                    base_url = "https://www.w3schools.com"
                    
            elif extra is None :
                continue 
           
                
                
            
            page_url = base_url + extra
            course_links.append(page_url)
            sub_page = requests.get(page_url)
            soup_page = BeautifulSoup(sub_page.content, 'html.parser')
            
            page_content = soup_page.find_all('div', class_ = 'w3-col l10 m12', id='main')
            page_examples = soup_page.find_all('div', class_ = 'w3-code notranslate')
            
            ##################################################################################################
            
            # Examples per page
            pg_example = []
            for example in page_examples:
                nw = []
                for i in list(example):
                    if str(i) == '<br/>':
                        pass
                    else:
                        nw.append(i)
                pg_example.append(nw)
            examples_per_page.append(pg_example)
                
            ##################################################################################################
            
            # Search for tables 
            tables = soup_page.find_all('table')

            data = []
            for table in tables:
                all_rows = table.find_all('tr')
                for row in all_rows:
                    row_list = row.find_all('td')
                    dataRow = []
                    for cells in row_list:
                        dataRow.append(cells.text)
                    data.append(dataRow)

            table_new = []
            row = ''
            for i in range(1, len(data)):
                if len(data[i]) == 0:
                    row = row[:-3]
                    table_new.append(row) 
                    row = ''
                for j in data[i]:
                    row += j + "|||"
                    # using separator for each column
                row = row[:-3]
                # using '&&&' separator for each row
                row += '&&&'
                #print(row)
            row = row[:-3]
            if len(row) !=0:
                if row[0] == '&':
                    row = row[3:]
            table_new.append(row)
            table_string = ''
            for each in table_new:
                table_string += each + '$$$'
            table_new = table_string[:-3]
            page_table.append(table_new)
            
            ##################################################################################################
            
            # Points for Unordered lists
            ul = soup_page.find_all('ul')
            unorder = []
            for i in ul:
                start_string = ''
                for j in i:
                    if j.text != '\n':
                        #print(j.text.replace('\n',''))
                        start_string += j.text.replace('\n','') + '@@@'
                start_string = start_string[:-3]
                unorder.append(start_string)
            if len(unorder) == 0:
                unorder = None
            else:
                unorder_string = ''
                for i in unorder:
                    unorder_string += i +'|||'
                unorder = unorder_string[:-3]
            unordered_lists.append(unorder)
            
                        
            ##################################################################################################
            #Points for ordered list.
            ol = soup_page.find_all('ol')
            order = []
            for i in ol:
                start_string = ''
                for j in i:
                    if j.text != '\n':
                        #print(j.text.replace('\n',''))
                        start_string += j.text.replace('\n','') + '@@@'
                start_string = start_string[:-3]
                order.append(start_string)
            
            if len(order) == 0:
                order = None
            else:
                order_string = ''
                for i in order:
                    order_string += i +'|||'
                order = order_string[:-3]
            ordered_lists.append(order)
            
            ##################################################################################################
            
            # Headings and Subheadings and Description
            
            list_headings_per_page = []
            for page_data in page_content:
                for j in str(page_data).split('\n'):
                    list_headings_per_page.append(j)
                page_paragraphs = page_data.find_all('p')
        
            content_per_page = ''
            for i in page_paragraphs:
                content_per_page += i.text + '|||'
            content_per_page = content_per_page[:-3].replace('\n', '')
            sub_page_content_list.append(content_per_page)
            
            prev = ''
            ul_ol_heading = ''
            for i in list_headings_per_page:
                if ('ul' in i) and (('h2' in prev) or ('h3' in prev)):
                    ul_ol_heading += prev[4:-5] + '|||'
                prev = i
            list_headings.append(ul_ol_heading[:-3])
#             time.sleep(5)      

default.asp
webpages_intro.asp
webpages_razor.asp
webpages_layout.asp
webpages_folders.asp
webpages_global.asp
webpages_forms.asp
webpages_objects.asp
webpages_files.asp
webpages_database.asp
webpages_helpers.asp
webpages_webgrid.asp
webpages_chart.asp
webpages_email.asp
webpages_security.asp
webpages_publish.asp
webpages_examples.asp
webpages_classes.asp
razor_intro.asp
razor_syntax.asp
razor_cs_variables.asp
razor_cs_loops.asp
razor_cs_logic.asp
razor_vb_variables.asp
razor_vb_loops.asp
razor_vb_logic.asp
asp_introduction.asp
asp_syntax.asp
asp_variables.asp
asp_procedures.asp
asp_conditionals.asp
asp_looping.asp
asp_inputforms.asp
asp_cookies.asp
asp_sessions.asp
asp_applications.asp
asp_incfiles.asp
asp_globalasa.asp
asp_ajax.asp
asp_send_email.asp
asp_examples.asp
asp_ref_vbscript_functions.asp
asp_ref_vbscript_keywords.asp
asp_ref_response.asp
asp_ref_request.asp
asp_ref_application.asp
asp_ref_session.asp
asp_ref_server.asp
asp_ref_error.asp
asp_ref_filesystem.asp
asp_ref_textst

In [11]:
length = len(course_links)
length

81

In [12]:
len(course_heading), len(course_subheading), len(course_links), len(sub_page_content_list), len(list_headings), len(unordered_lists),len(ordered_lists),  len(page_table), len(examples_per_page)

(81, 81, 81, 81, 81, 81, 81, 81, 81)

In [13]:
from csv import writer

# writing the file
with open('asp_w3school.csv', 'w', encoding='utf8', newline='') as f:
    the_writer = writer(f)
    header = ['Heading', 'Sub-Heading', 'Links', 'Description','List Headings', 
              'Unordered Lists', 'Ordered Lists','Table Data', 'Examples']
    the_writer.writerow(header)
    for i in range(length):
        row = [course_heading[i], course_subheading[i], course_links[i], sub_page_content_list[i], list_headings[i], 
               unordered_lists[i], ordered_lists[i], page_table[i], examples_per_page[i]]
        the_writer.writerow(row)

# reading the file
file = pd.read_csv('asp_w3school.csv')
dataset = pd.DataFrame(file)

# viewing first 5 rows
dataset.head(5)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples
0,ASP Tutorials,ASP HOME,https://www.w3schools.com/asp/default.asp,ASP stands for Active Server Pages|||ASP is a ...,NaN,Classic ASP@@@ASP.NET Web Forms@@@ASP.NET MVC@...,NaN,NaN,[]
1,WP Tutorial,WebPages Intro,https://www.w3schools.com/asp/webpages_intro.asp,"Our ""Run Example"" tool displays the ASP.NET co...","Easy Learning with ""Run Example""","Easy to learn, understand, and use@@@Uses an S...",NaN,NaN,"[['\n<html>', '\n<body>', '\n\xa0\xa0\xa0\xa0\..."
2,WP Tutorial,WebPages Razor,https://www.w3schools.com/asp/webpages_razor.asp,ASP.NET Web Pages use Razor markup with C# or ...,Razor Syntax for C#|||Razor Syntax for VB,C# code blocks are enclosed in @{ ... }@@@Inli...,NaN,NaN,"[['\n<!DOCTYPE html>', '\n', '\n<html lang=""en..."
3,WP Tutorial,WebPages Layout,https://www.w3schools.com/asp/webpages_layout.asp,With Web Pages it is easy to create a web site...,NaN,Every page have the same header@@@Every page h...,NaN,NaN,"[['\n<html>', '\n<body>', '\n', <span class=""m..."
4,WP Tutorial,WebPages Folders,https://www.w3schools.com/asp/webpages_folders...,This chapter is about folders and folder paths...,er.MapPath Method,About Logical and Physical folder structures@@...,NaN,Server name|||w3schools&&&Virtual path|||/html...,[]


In [14]:
# Re-checking the dataset and removing cells that have no relevant data
for i in range(len(dataset['Table Data'])):
    try:
        if len(dataset['Table Data'][i]) == 4:
            dataset['Table Data'][i] = None
        if len(dataset['Examples'][i]) == 2:
            dataset['Examples'][i] = None
    except TypeError:
        pass

In [15]:
from csv import writer

# writing the file
with open('asp_w3school.csv', 'w', encoding='utf8', newline='') as f:
    the_writer = writer(f)
    header = ['Heading', 'Sub-Heading', 'Links', 'Description','List Headings', 
              'Unordered Lists', 'Ordered Lists','Table Data', 'Examples']
    the_writer.writerow(header)
    for i in range(length):
        row = [course_heading[i], course_subheading[i], course_links[i], sub_page_content_list[i], list_headings[i], 
               unordered_lists[i], ordered_lists[i], page_table[i], examples_per_page[i]]
        the_writer.writerow(row)

# reading the file
file = pd.read_csv('asp_w3school.csv')
dataset = pd.DataFrame(file)

# viewing first 5 rows
dataset.head(5)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples
0,ASP Tutorials,ASP HOME,https://www.w3schools.com/asp/default.asp,ASP stands for Active Server Pages|||ASP is a ...,NaN,Classic ASP@@@ASP.NET Web Forms@@@ASP.NET MVC@...,NaN,NaN,[]
1,WP Tutorial,WebPages Intro,https://www.w3schools.com/asp/webpages_intro.asp,"Our ""Run Example"" tool displays the ASP.NET co...","Easy Learning with ""Run Example""","Easy to learn, understand, and use@@@Uses an S...",NaN,NaN,"[['\n<html>', '\n<body>', '\n\xa0\xa0\xa0\xa0\..."
2,WP Tutorial,WebPages Razor,https://www.w3schools.com/asp/webpages_razor.asp,ASP.NET Web Pages use Razor markup with C# or ...,Razor Syntax for C#|||Razor Syntax for VB,C# code blocks are enclosed in @{ ... }@@@Inli...,NaN,NaN,"[['\n<!DOCTYPE html>', '\n', '\n<html lang=""en..."
3,WP Tutorial,WebPages Layout,https://www.w3schools.com/asp/webpages_layout.asp,With Web Pages it is easy to create a web site...,NaN,Every page have the same header@@@Every page h...,NaN,NaN,"[['\n<html>', '\n<body>', '\n', <span class=""m..."
4,WP Tutorial,WebPages Folders,https://www.w3schools.com/asp/webpages_folders...,This chapter is about folders and folder paths...,er.MapPath Method,About Logical and Physical folder structures@@...,NaN,Server name|||w3schools&&&Virtual path|||/html...,[]


In [17]:
# Re-checking the dataset and removing cells that have no relevant data
for i in range(len(dataset['Table Data'])):
    try:
        if len(dataset['Table Data'][i]) == 4:
            dataset['Table Data'][i] = None
        if len(dataset['Examples'][i]) == 2:
            dataset['Examples'][i] = None
    except TypeError:
        pass

In [18]:
dataset = dataset.replace(np.nan, '', regex=True)

In [19]:
# Making a separate column for those heading which are not associated with the Lists
new_col = [None for i in range(len(course_links))]
dataset['Other Headings'] = pd.Series(new_col)
for i in range(len(dataset['Unordered Lists'])):
    if dataset['Unordered Lists'][i] == '':
        dataset['Other Headings'][i] = dataset['List Headings'][i]
        dataset['List Headings'][i] = ''

In [20]:
# Removing the Null values and keeping the cells empty
dataset = dataset.replace(np.nan, '', regex=True)
dataset.head(10)

,Heading,Sub-Heading,Links,Description,List Headings,Unordered Lists,Ordered Lists,Table Data,Examples,Other Headings
0,ASP Tutorials,ASP HOME,https://www.w3schools.com/asp/default.asp,ASP stands for Active Server Pages|||ASP is a ...,,Classic ASP@@@ASP.NET Web Forms@@@ASP.NET MVC@...,,,[],
1,WP Tutorial,WebPages Intro,https://www.w3schools.com/asp/webpages_intro.asp,"Our ""Run Example"" tool displays the ASP.NET co...","Easy Learning with ""Run Example""","Easy to learn, understand, and use@@@Uses an S...",,,"[['\n<html>', '\n<body>', '\n\xa0\xa0\xa0\xa0\...",
2,WP Tutorial,WebPages Razor,https://www.w3schools.com/asp/webpages_razor.asp,ASP.NET Web Pages use Razor markup with C# or ...,Razor Syntax for C#|||Razor Syntax for VB,C# code blocks are enclosed in @{ ... }@@@Inli...,,,"[['\n<!DOCTYPE html>', '\n', '\n<html lang=""en...",
3,WP Tutorial,WebPages Layout,https://www.w3schools.com/asp/webpages_layout.asp,With Web Pages it is easy to create a web site...,,Every page have the same header@@@Every page h...,,,"[['\n<html>', '\n<body>', '\n', <span class=""m...",
4,WP Tutorial,WebPages Folders,https://www.w3schools.com/asp/webpages_folders...,This chapter is about folders and folder paths...,er.MapPath Method,About Logical and Physical folder structures@@...,,Server name|||w3schools&&&Virtual path|||/html...,,
5,WP Tutorial,WebPages Global,https://www.w3schools.com/asp/webpages_global.asp,This chapter is about the global pages AppStar...,,,,,[],
6,WP Tutorial,WebPages Forms,https://www.w3schools.com/asp/webpages_forms.asp,A form is a section of an HTML document where ...,,,,,"[['\n<html>', '\n<body>\xa0', '\n', <span clas...",
7,WP Tutorial,WebPages Objects,https://www.w3schools.com/asp/webpages_objects...,Web Pages is much often about Objects.|||You ...,&lt;h1&gt;Welcome to W3Schools&lt;/h1&gt; <br/...,Page.Title@@@Page.Version@@@Page.anythingyoulike,,href|||Builds a URL using the specified parame...,"[['\n', <span class=""marked"">@{<br/>Layout=""~/...",
8,WP Tutorial,WebPages Files,https://www.w3schools.com/asp/webpages_files.asp,This chapter is about working with text files....,,How to read and display data from a text file,,,"[['\nGeorge,Lucas', '\nSteven,Spielberg', '\nA...",
9,WP Tutorial,WebPages Databases,https://www.w3schools.com/asp/webpages_databas...,This chapter is about working with databases.|...,,Create a web page to list data from a database,,"Database.Execute(SQLstatement [,\nparameters])...","[['\n', <span class=""marked"">@{<br/>\nvar db =...",
